In [1]:
from tf_version import SSD300v2 as tf_ssd
from torch_version import SSD300v2 as torch_ssd
from torchsummary import summary

import torch
import tensorflow as tf
import numpy as np
tf.__version__

2021-08-29 02:49:54.674946: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


'2.3.0'

In [2]:
torch_model = torch_ssd((300, 300, 3), num_classes=21)
torch_model.float()
torch_model.eval()
#with torch.no_grad():
#    summary(torch_model.to('cuda'), (3, 300, 300))

SSD300v2(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3z): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (

In [3]:
tf_model = tf_ssd((300, 300, 3), num_classes=21)
#tf_model.summary()

2021-08-29 02:49:55.891839: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-29 02:49:55.903670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 02:49:55.904056: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1070 computeCapability: 6.1
coreClock: 1.7465GHz coreCount: 15 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 238.66GiB/s
2021-08-29 02:49:55.904071: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-29 02:49:55.905309: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-08-29 02:49:55.906384: I tensorflow/stream_executor/platform/


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [4]:
tf_model.load_weights('weights_SSD300.hdf5', by_name=True)
torch_model.load_state_dict(torch.load('SSD300v2_torch.pth'))

<All keys matched successfully>

In [5]:
frame = (np.random.randn(300, 300, 3) * 255).astype(np.float32)
frame_torch = np.copy(frame.transpose([2, 0, 1]))

In [6]:
with torch.no_grad():
    torch_out = torch_model(torch.tensor(frame_torch)[None]).numpy()
torch_out.shape, torch_out

/home/thea/2021/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


((1, 7308, 33),
 array([[[ 1.20047498e-02, -5.93187094e-01, -2.88489521e-01, ...,
           1.00000000e-01,  2.00000000e-01,  2.00000000e-01],
         [ 9.95802879e-03, -7.78673440e-02, -2.67096728e-01, ...,
           1.00000000e-01,  2.00000000e-01,  2.00000000e-01],
         [-3.59781444e-01, -2.60872811e-01, -5.65625250e-01, ...,
           1.00000000e-01,  2.00000000e-01,  2.00000000e-01],
         ...,
         [ 5.39384186e-01, -1.40195526e-03,  7.78938353e-01, ...,
           1.00000000e-01,  2.00000000e-01,  2.00000000e-01],
         [ 3.24449658e-01, -2.26651236e-01, -1.31434584e+00, ...,
           1.00000000e-01,  2.00000000e-01,  2.00000000e-01],
         [ 7.59095073e-01,  5.51224574e-02,  1.93609285e+00, ...,
           1.00000000e-01,  2.00000000e-01,  2.00000000e-01]]]))

In [7]:
tf_out = tf_model.predict(frame[None])[1]
tf_out.shape, tf_out

2021-08-29 02:49:57.881335: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-08-29 02:49:57.972552: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


((1, 7308, 33),
 array([[[ 1.2004881e-02, -5.9318697e-01, -2.8848940e-01, ...,
           1.0000000e-01,  2.0000000e-01,  2.0000000e-01],
         [ 9.9576050e-03, -7.7867121e-02, -2.6709747e-01, ...,
           1.0000000e-01,  2.0000000e-01,  2.0000000e-01],
         [-3.5978121e-01, -2.6087278e-01, -5.6562555e-01, ...,
           1.0000000e-01,  2.0000000e-01,  2.0000000e-01],
         ...,
         [ 5.3938448e-01, -1.4018212e-03,  7.7893865e-01, ...,
           1.0000000e-01,  2.0000000e-01,  2.0000000e-01],
         [ 3.2444954e-01, -2.2665162e-01, -1.3143457e+00, ...,
           1.0000000e-01,  2.0000000e-01,  2.0000000e-01],
         [ 7.5909531e-01,  5.5122502e-02,  1.9360934e+00, ...,
           1.0000000e-01,  2.0000000e-01,  2.0000000e-01]]], dtype=float32))

In [8]:
try:
    a, b = np.max(np.abs(torch_out - tf_out)), np.mean(np.abs(torch_out - tf_out))
except:
    a, b = 'a', 'b'
a, b

(7.62939453125e-06, 9.265810422026834e-08)

In [9]:
try:
    a, b = np.max(np.abs(torch_out - tf_out.transpose([0, 3, 1, 2]))), np.mean(torch_out - tf_out.transpose([0, 3, 1, 2]))
except:
    a, b = 'a', 'b'
a, b

('a', 'b')

In [10]:
tf_model.get_layer('conv4_3_norm_mbox_conf').weights[0].numpy().transpose([3, 2, 0, 1])

array([[[[-0.00292147,  0.00901721,  0.01918894],
         [ 0.00808298,  0.00191519,  0.00481817],
         [ 0.00608263, -0.00210195,  0.00298199]],

        [[ 0.00783363, -0.00370542,  0.02037198],
         [ 0.02561151,  0.01210603,  0.01751088],
         [ 0.00153678,  0.01628629,  0.01919375]],

        [[ 0.01239109,  0.01266858,  0.01743064],
         [-0.0031695 ,  0.0281014 ,  0.02572742],
         [ 0.01256821,  0.0180959 ,  0.00611735]],

        ...,

        [[ 0.00086207,  0.01278962,  0.01083817],
         [ 0.01484153,  0.04083962,  0.00452134],
         [ 0.01683346,  0.00717383,  0.02132946]],

        [[ 0.01155616,  0.00960493,  0.02923486],
         [ 0.01148837,  0.01271693,  0.01997035],
         [ 0.01456796,  0.01266645,  0.02131977]],

        [[-0.00422126, -0.00311682, -0.01185097],
         [ 0.00977793, -0.00636645, -0.01513981],
         [-0.00393042,  0.0003102 ,  0.00475151]]],


       [[[ 0.00212998, -0.01229266, -0.00330767],
         [ 0.01711095,

In [11]:
torch_model.conv4_3_norm_mbox_conf.weight

Parameter containing:
tensor([[[[-0.0029,  0.0090,  0.0192],
          [ 0.0081,  0.0019,  0.0048],
          [ 0.0061, -0.0021,  0.0030]],

         [[ 0.0078, -0.0037,  0.0204],
          [ 0.0256,  0.0121,  0.0175],
          [ 0.0015,  0.0163,  0.0192]],

         [[ 0.0124,  0.0127,  0.0174],
          [-0.0032,  0.0281,  0.0257],
          [ 0.0126,  0.0181,  0.0061]],

         ...,

         [[ 0.0009,  0.0128,  0.0108],
          [ 0.0148,  0.0408,  0.0045],
          [ 0.0168,  0.0072,  0.0213]],

         [[ 0.0116,  0.0096,  0.0292],
          [ 0.0115,  0.0127,  0.0200],
          [ 0.0146,  0.0127,  0.0213]],

         [[-0.0042, -0.0031, -0.0119],
          [ 0.0098, -0.0064, -0.0151],
          [-0.0039,  0.0003,  0.0048]]],


        [[[ 0.0021, -0.0123, -0.0033],
          [ 0.0171,  0.0152,  0.0068],
          [ 0.0177,  0.0129, -0.0017]],

         [[-0.0205, -0.0109, -0.0152],
          [-0.0033, -0.0196,  0.0058],
          [-0.0038, -0.0201, -0.0134]],

         